In [72]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import math
import six
import sys
sys.modules['sklearn.externals.six'] = six
import statistics

import mlrose

import yfinance as yf
yf.pdr_override()

dataset = pd.read_csv('db/acoes1.csv')
dataset.drop(columns=["BOVA11.SA"], inplace=True)
dataset


,Date,VALE3.SA,KEPL3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,DIRR3.SA,MILS3.SA,...,CLSC4.SA,CMIG4.SA,ITSA4.SA,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA
0,2023-03-01,84.727066,9.253362,10.783572,30.792067,26.862852,17.323414,15.894750,14.850000,10.293840,...,45.495796,9.862920,7.746500,37.020172,13.199746,41.843403,10.179686,20.357569,46.770000,3.820549
1,2023-03-02,84.537094,9.882277,10.429856,30.283728,26.908350,17.516972,15.953078,14.970000,10.333017,...,46.216782,9.777896,7.575413,35.529869,12.823399,41.860493,10.266940,19.826502,46.700001,3.810828
2,2023-03-03,84.822060,9.868402,10.401176,30.735586,26.771854,17.178246,16.040573,14.810000,10.421165,...,45.828560,9.711763,7.594422,35.928547,13.153851,41.869038,10.199076,20.679426,48.200001,3.849714
3,2023-03-06,81.830009,9.822159,10.458536,30.660275,25.916466,17.555681,16.409990,15.080000,10.391782,...,46.207535,9.862920,7.756005,36.744892,13.291539,42.133923,10.625652,20.888636,47.779999,3.810828
4,2023-03-07,81.051132,9.341226,10.429856,29.916597,25.561569,17.652460,16.409990,15.260000,10.793349,...,45.449581,9.947946,7.727491,37.589710,13.429228,41.740868,10.712907,20.196640,47.750000,3.771942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2023-10-19,64.419998,10.420000,18.580000,38.759998,21.629999,12.340000,21.959999,16.750000,10.420000,...,62.119999,11.780000,8.748500,49.500000,15.470000,36.689999,11.000000,38.340000,55.980000,4.650000
161,2023-10-20,62.680000,10.460000,18.290001,38.849998,21.450001,12.220000,22.020000,16.709999,10.600000,...,61.259998,11.810000,8.740000,49.340000,15.340000,36.270000,11.150000,37.849998,54.900002,4.550000
162,2023-10-23,62.560001,10.600000,18.330000,38.000000,21.760000,12.400000,22.860001,17.049999,10.630000,...,63.000000,11.910000,8.730000,49.270000,15.770000,36.500000,11.310000,35.349998,54.400002,4.480000
163,2023-10-24,63.990002,10.750000,18.150000,40.799999,21.549999,12.380000,23.330000,17.230000,10.800000,...,62.889999,11.930000,8.780000,49.040001,15.620000,36.849998,11.600000,35.880001,52.509998,4.450000


In [73]:
dinheiro_total = 5000
taxa_selic_2010 = 12.5
taxa_selic_2011 = 9.25
taxa_selic_2012 = 7.25
taxa_selic_2013 = 7.75
taxa_selic_2014 = 9.5
taxa_selic_2015 = 14.25
taxa_selic_2016 = 14.25
taxa_selic_2017 = 14.25
taxa_selic_2018 = 6.5
taxa_selic_2019 = 5.25
taxa_selic_2020 = 2.25
taxa_selic_2021 = 7.75
taxa_selic_2022 = 11.75
taxa_selic_2023 = 12.5

valor_2010 = dinheiro_total + (dinheiro_total * taxa_selic_2010 / 100)
valor_2011 = valor_2010 + (dinheiro_total * taxa_selic_2011 / 100)
valor_2012 = valor_2011 + (dinheiro_total * taxa_selic_2012 / 100)
valor_2013 = valor_2012 + (dinheiro_total * taxa_selic_2013 / 100)
valor_2014 = valor_2013 + (dinheiro_total * taxa_selic_2014 / 100)
valor_2015 = valor_2014 + (dinheiro_total * taxa_selic_2015 / 100)
valor_2016 = valor_2015 + (dinheiro_total * taxa_selic_2016 / 100)
valor_2017 = valor_2016 + (dinheiro_total * taxa_selic_2017 / 100)
valor_2018 = valor_2017 + (dinheiro_total * taxa_selic_2018 / 100)
valor_2019 = valor_2018 + (dinheiro_total * taxa_selic_2019 / 100)
valor_2020 = valor_2019 + (dinheiro_total * taxa_selic_2020 / 100)
valor_2021 = valor_2020 + (dinheiro_total * taxa_selic_2021 / 100)
valor_2022 = valor_2021 + (dinheiro_total * taxa_selic_2022 / 100)
valor_2023 = valor_2022 + (dinheiro_total * taxa_selic_2023 / 100)
valor_2023
taxa_selic_historico = np.array([taxa_selic_2010, taxa_selic_2011, taxa_selic_2012, taxa_selic_2013, taxa_selic_2014, taxa_selic_2015, taxa_selic_2016, taxa_selic_2017, taxa_selic_2018, taxa_selic_2019, taxa_selic_2020, taxa_selic_2021, taxa_selic_2022, taxa_selic_2023])
dataset_original = dataset.copy()
sem_risco = taxa_selic_historico.mean() / 100
sem_risco

# (dataset['taxa de retorno'].mean() - taxa_selic_historico.mean()/100)/ dataset['taxa de retorno'].std() * math.sqrt(246) #sharp ratio validando historico da selic


0.09642857142857142

In [74]:
len(dataset.columns[1:])
dataset.columns
dataset_original

,Date,VALE3.SA,KEPL3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,DIRR3.SA,MILS3.SA,...,CLSC4.SA,CMIG4.SA,ITSA4.SA,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA
0,2023-03-01,84.727066,9.253362,10.783572,30.792067,26.862852,17.323414,15.894750,14.850000,10.293840,...,45.495796,9.862920,7.746500,37.020172,13.199746,41.843403,10.179686,20.357569,46.770000,3.820549
1,2023-03-02,84.537094,9.882277,10.429856,30.283728,26.908350,17.516972,15.953078,14.970000,10.333017,...,46.216782,9.777896,7.575413,35.529869,12.823399,41.860493,10.266940,19.826502,46.700001,3.810828
2,2023-03-03,84.822060,9.868402,10.401176,30.735586,26.771854,17.178246,16.040573,14.810000,10.421165,...,45.828560,9.711763,7.594422,35.928547,13.153851,41.869038,10.199076,20.679426,48.200001,3.849714
3,2023-03-06,81.830009,9.822159,10.458536,30.660275,25.916466,17.555681,16.409990,15.080000,10.391782,...,46.207535,9.862920,7.756005,36.744892,13.291539,42.133923,10.625652,20.888636,47.779999,3.810828
4,2023-03-07,81.051132,9.341226,10.429856,29.916597,25.561569,17.652460,16.409990,15.260000,10.793349,...,45.449581,9.947946,7.727491,37.589710,13.429228,41.740868,10.712907,20.196640,47.750000,3.771942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2023-10-19,64.419998,10.420000,18.580000,38.759998,21.629999,12.340000,21.959999,16.750000,10.420000,...,62.119999,11.780000,8.748500,49.500000,15.470000,36.689999,11.000000,38.340000,55.980000,4.650000
161,2023-10-20,62.680000,10.460000,18.290001,38.849998,21.450001,12.220000,22.020000,16.709999,10.600000,...,61.259998,11.810000,8.740000,49.340000,15.340000,36.270000,11.150000,37.849998,54.900002,4.550000
162,2023-10-23,62.560001,10.600000,18.330000,38.000000,21.760000,12.400000,22.860001,17.049999,10.630000,...,63.000000,11.910000,8.730000,49.270000,15.770000,36.500000,11.310000,35.349998,54.400002,4.480000
163,2023-10-24,63.990002,10.750000,18.150000,40.799999,21.549999,12.380000,23.330000,17.230000,10.800000,...,62.889999,11.930000,8.780000,49.040001,15.620000,36.849998,11.600000,35.880001,52.509998,4.450000


In [75]:
def fitness_function(solucao):
    a = []
    

    for j in range(12,24):
        
        dataset = pd.read_csv(f'db/acoes20{j}.csv')
        pesos = solucao / solucao.sum()
        dataset.drop(columns=["BOVA11.SA"], inplace=True)

        for i in dataset.columns[1:]:
            dataset[i] = (dataset[i] / dataset[i][0]) # normalizador das colunas

        for i, acao in enumerate(dataset.columns[1:]):
            dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

        dataset.drop(labels = ['Date'], axis = 1, inplace=True)
        dataset['soma valor'] = dataset.sum(axis = 1)
        
        
        dataset['taxa retorno'] = 0.0
        dataset["taxa retorno"] = dataset["soma valor"].pct_change()
        risk_free = sem_risco/252
        dataset["retorno Calc"] = dataset["taxa retorno"] - risk_free
        sharpe_ratio = np.sqrt(252) * (dataset["retorno Calc"].mean()/dataset["retorno Calc"].std())  
        a.append(sharpe_ratio)
        
    peso_valor = [1,1,1,1,1,1,2,2,3,3,4,5]
    np.multiply(a, peso_valor)
    sharpe_ratio = statistics.mean(a)/25   
    return sharpe_ratio


# def fitness_function(solucao):
  
#   dataset = dataset_original.copy()
#   pesos = solucao / solucao.sum()

#   for i in dataset.columns[1:]:
#     dataset[i] = (dataset[i] / dataset[i][0]) # normalizador das colunas

#   for i, acao in enumerate(dataset.columns[1:]):
#     dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

#   dataset.drop(labels = ['Date'], axis = 1, inplace=True)
#   dataset['soma valor'] = dataset.sum(axis = 1)
  
  
#   dataset['taxa retorno'] = 0.0
#   dataset["taxa retorno"] = dataset["soma valor"].pct_change()
#   risk_free = sem_risco/252
#   dataset["retorno Calc"] = dataset["taxa retorno"] - risk_free
#   sharpe_ratio = np.sqrt(252) * (dataset["retorno Calc"].mean()/dataset["retorno Calc"].std())  

#   return sharpe_ratio


In [76]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0, melhores_pesos = []):
    if seed != 0:
        np.random.seed(seed)

    dataset = dataset.copy()

    if len(melhores_pesos) > 0:
        pesos = melhores_pesos
    else:  
        pesos = np.random.random(len(dataset.columns) - 1)
    #print(pesos, pesos.sum())
        pesos = pesos / pesos.sum()
    #print(pesos, pesos.sum())
    
 
    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['SOMA VALOR'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']

    dataset.drop(columns=['Date'], inplace=True)
    dataset['taxa de retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa de retorno'][i] = ((dataset['SOMA VALOR'][i] / dataset['SOMA VALOR'][i-1]) - 1 )*100

    acoes_pesos = pd.DataFrame(data= {'Ações': colunas, 'Pesos': pesos*100})

    return dataset, datas, acoes_pesos , dataset.loc[len(dataset)-1]['SOMA VALOR']

In [77]:
np.random.seed(10)
pesos = np.random.random(len(dataset_original.columns) - 1)
pesos = pesos / pesos.sum()
pesos

array([0.06996354, 0.00188233, 0.0574758 , 0.06792113, 0.04521766,
       0.02039044, 0.01796552, 0.06898482, 0.0153394 , 0.00801297,
       0.06216636, 0.08647865, 0.00035813, 0.046459  , 0.07370973,
       0.05555989, 0.06546766, 0.02647496, 0.08324777, 0.06481643,
       0.04921212, 0.0128957 ])

In [78]:
def visualiza_alocacao(solucao):
  colunas = dataset_original.columns[1:]
  for i in range(len(solucao)):
    print(colunas[i], solucao[i] * 100)
visualiza_alocacao(pesos)    

VALE3.SA 6.996353716330795
KEPL3.SA 0.18823297326368424
VULC3.SA 5.7475801042961985
LEVE3.SA 6.792112816031783
GGBR4.SA 4.52176590706652
LREN3.SA 2.0390441512331403
ALSO3.SA 1.7965522795542441
DIRR3.SA 6.898482390074899
MILS3.SA 1.5339395362804158
RANI3.SA 0.8012965717845643
CAMB3.SA 6.216636043802957
CSUD3.SA 8.647865370911916
CLSC4.SA 0.03581320950956965
CMIG4.SA 4.645899570686953
ITSA4.SA 7.370973063736465
BBAS3.SA 5.555989018861558
TASA4.SA 6.546765656379767
SLCE3.SA 2.6474958657983443
B3SA3.SA 8.324777054586187
PETR4.SA 6.481642856854627
SUZB3.SA 4.921211870245791
KLBN4.SA 1.2895699727096261


In [79]:
fitness = mlrose.CustomFitness(fitness_function)

In [80]:
problema_maximizacao = mlrose.ContinuousOpt(length=22, fitness_fn=fitness,
                                            maximize = True, min_val = 0, max_val = 1)

In [81]:
problema_minimizacao = mlrose.ContinuousOpt(length=22, fitness_fn=fitness,
                                            maximize = False, min_val = 0, max_val = 1)

In [82]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema_maximizacao, random_state = 1) ## hill climb
melhor_solucao, melhor_custo

(array([0.        , 0.62032449, 0.30011437, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.83881673,
        0.        , 0.6852195 , 0.30445225, 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.49810149,
        0.        , 0.        ]),
 0.03085181941282382)

In [83]:
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_solucao.sum()

(array([0.        , 0.14606081, 0.07066455, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.19750672,
        0.        , 0.16134091, 0.07168594, 0.        , 0.        ,
        0.        , 0.        , 0.23545873, 0.        , 0.11728234,
        0.        , 0.        ]),
 1.0)

In [84]:
visualiza_alocacao(melhor_solucao)

VALE3.SA 0.0
KEPL3.SA 14.606081488878472
VULC3.SA 7.066454832763578
LEVE3.SA 0.0
GGBR4.SA 0.0
LREN3.SA 0.0
ALSO3.SA 0.0
DIRR3.SA 0.0
MILS3.SA 0.0
RANI3.SA 19.75067194768176
CAMB3.SA 0.0
CSUD3.SA 16.134091054550453
CLSC4.SA 7.168593882816861
CMIG4.SA 0.0
ITSA4.SA 0.0
BBAS3.SA 0.0
TASA4.SA 0.0
SLCE3.SA 23.545872592955106
B3SA3.SA 0.0
PETR4.SA 11.728234200353775
SUZB3.SA 0.0
KLBN4.SA 0.0


In [85]:
_, _, _, soma_valor = alocacao_ativos(dataset, 5000, melhores_pesos=melhor_solucao)
soma_valor

6705.952761599288

In [86]:
## Simulated annealing
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema_maximizacao, random_state = 1) 
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo

In [ ]:
visualiza_alocacao(melhor_solucao)

In [ ]:
_, _, _, soma_valor = alocacao_ativos(dataset, 5000, melhores_pesos=melhor_solucao)
soma_valor

In [ ]:
#algoritmo genético 
problema_maximizacao_ag = mlrose.ContinuousOpt(length = 22, fitness_fn = fitness, 
                                               maximize = True, min_val = 0.01, max_val = 1)
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema_maximizacao_ag, random_state = 1, pop_size=8000, mutation_prob= 0.1, max_attempts = 30)
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo


In [ ]:
visualiza_alocacao(melhor_solucao)

In [ ]:
_, _, _, soma_valor = alocacao_ativos(dataset, 5000, melhores_pesos=melhor_solucao)
soma_valor